In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('../../training/kjbible.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '’', '\ufeff']


In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 66, 54, 51,  1, 69, 47, 58],
        [58, 65,  1, 61, 52,  1, 53, 61],
        [67, 66,  0, 61, 52,  1, 66, 54],
        [66, 61,  1, 26, 58, 51, 47, 72]], device='cuda:0')
targets:
tensor([[66, 54, 51,  1, 69, 47, 58, 58],
        [65,  1, 61, 52,  1, 53, 61, 58],
        [66,  0, 61, 52,  1, 66, 54, 71],
        [61,  1, 26, 58, 51, 47, 72, 47]], device='cuda:0')


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)




lLd.’Dzp.qq2AP99eiPWWl)’Pp)8c::o:’y0-xy5yu-Ifn.S ;WCjLqr’5
y3CdF-9v72-e’y88yl TJ(FPAf8yNqLJ?ih0rV6’-9);Fxyi 6Ydp1DTqKlxpL?lW﻿YRjy1;6U’Yd!,.8dB(NM(zSvRj:OpYqcja﻿vcAEo:bi6!
l2VVMtNF9SNei u,sK.’yH5;-n.RPE4voQpLYd!r:bD*qtkNFP5kJfzhjleqv1dE4fViPIxWAr8B6kzR;75jPKTKU2CgIs2f-oB!:1JiYd7.IjK’Fx8’DzE4:8Bwbw﻿’08r.NolmiO!4waQNrCcbI*5RWYMoBI8n!KF-xv7)kM;?BFeRL6p67.RiqRzwNyI-J(Afb(ZHid7.U9BQ7.E:vZZB5kDuZ7E!YD9.F0ZoDzcDpl5jfVVVRD;SrUnNFzUQtS*sf8﻿SolF’-RQbHyNvZWCcIMj
gJVVbTcw’VQvIJ﻿NS2amcw 
.0NYiOxckvgMJiORwbWEf


## optimizer

In [7]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 5.051, val loss: 5.051
step: 250, train loss: 4.966, val loss: 4.984
step: 500, train loss: 4.925, val loss: 4.913
step: 750, train loss: 4.833, val loss: 4.841
5.163896083831787


In [8]:
context = torch.zeros((1,1),dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
generated_chars

'\nIf8rewqvHMtk:EWuzdE3yFSmMpSrOKfbrWOh HiYhNiFRUj?fj)R,7bMjDa\nLcSnGhuJ8,7C?bMth0h0n;OeBO3Jcw6gz)1D*aLz;Fx86zv:lmbeL-0-q,7CYHj?fGKAPU!S!f!5\nk\ufeff\ufeffEW;p1YgwNF)r,Idy;72a??1oF-,.5(F:58-?Ydd!,V’Yd!:bML-yi97H3Cj(uKvcbvO’-Fh9yipd3x(?9Oqh0)naNYdk5exsFx7\ufeff’DzwB\ufeffy;8SOcwaJ7)hN9Klklm4u29ol)6.G4Sy\nQzVliamM?irIZz’(ombMD’Pys 9Lq:\ufeffj)it-jGIMKOr’Da.0n.FcwBymb;fPo90\ufeffySo:vc\nSr*pl7)qRP-2\nAGDZL\nvIjyW:osIfCdehJ1mO1(Q4*(Ll5k?y?WQVVoJmKtNy4:\ufeff047F4Yx0n5;’-uZW9R46nuTo3es;Ti\ufeffJU’DZ8R,7.HQaZjWEli z2vI(*u3KOB,M;558cAuB65jMJG)iJir,xx'

In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


CTJ1oj1DIJEWL
KNylyFxJr,T1dF97.bc2Z1Itywq
EhA!’ECTi
2UQQGo’-:R2vH MDK
p1iHkvKiW(mSx;r8bR*IEmOxb9QhSj9T1J;NiefWUplK8lmmM;QqG4*IrksHZd16i7.liDWEOKfZZxpdMKr.0-n﻿H2
lEkiCP,77?lBk2qhf1oo:w﻿’DYS8c(RPwymlh0re﻿yHio D?lor;glKfm7.1kmFF-?iJ﻿R-jAIWBMQpvKNVVyaQv
OGh)0857..’YMG4FSZGVGHc2xLYdp(LP-OMITiGhd!!eirIl8.8.ddjplYM!jj?BiS qA5kEL-9B1??6)qx(To6IoQ!sF6B97.GH’?m0Qc
uPZ!3C861D)wDvHhZqR96pD*:JseVMjq1
l25*w﻿f1pZvI)FN-x*Dua.0rWq3VVr8r’-2x8-xjtsuq0Kihov﻿R*
Zey1fMRP;p.6jdefeW.)oLJIqyrW8?I(MGKiGvn5*OimQ3VV69e1DDO
